### Build, test and push lambda container

In [ ]:
ecr_repo_name = "event_ai_lambda_image"
lambda_function_name = "event_ai_debug"
aws_region = "eu-west-1"

In [ ]:
!docker build -t {ecr_repo_name} .

In [ ]:
!docker run --rm --entrypoint python {ecr_repo_name} lambda_function.py

In [ ]:
import boto3
account_id = boto3.client("sts").get_caller_identity()["Account"]
print(account_id)

In [ ]:
ecr_url = f"{account_id}.dkr.ecr.{aws_region}.amazonaws.com/{ecr_repo_name}"
!aws ecr create-repository --repository-name {ecr_repo_name} --region {aws_region} || echo "Repo already exists"
!aws ecr get-login-password --region {aws_region} | \
    docker login --username AWS --password-stdin {ecr_url}

!docker tag {ecr_repo_name}:latest {ecr_url}:latest
!docker push {ecr_url}:latest


In [ ]:
!docker run --rm -it \
  -v ~/.aws:/root/.aws \
  -v "$PWD":/tmp \
  --entrypoint python \
  event_ai_lambda_image:latest lambda_function.py